## Viral taxonomy on pig vOTUs

in sourmash/viral_taxonomy

Do with both ICTV + genbank 

Steps:
1) run a fastmultigather of all vOTUs against a viral taxonomy db. 
all vOTUs in: sig_files/signatures_concat/allvOTUs.zip
2) do at different scales, 100 and 1000, see what the difference is between the 2. 
3) Use snakefile to go from fastmultigather to taxonomy. 

Compare to vcontact2 output and genomad output

### Sourmash:
- genbank
- ictv
- scaled 100 and 1000


In [ ]:
sourmash scripts fastmultigather \
sig.txt \
GCA_018585365.k31.rocksdb \
-k 31 -t 1000 -s 1000


In [ ]:
# symlink the dbs
# symlink
ln -s /path/to/db/ .

# genbank db and taxonomy file:
/home/ntpierce/2023-spillover/output.genbank-viral/genbank.2023-05.viral.dna.zip
/home/ntpierce/database-releases/genbank-2023.05/viral.lineages.csv

# ICTV dbs and tax file
/home/ntpierce/2023-spillover/output.vmr/
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.taxonomy.csv

# try the rocksdbs

## ICTV
- run against ictv tax
- use scale 100
- use scale 1000

For now just use a bp treshold of 0, can always adjust later. Can subset tax results on bp_match_at_rank

In [ ]:
# run fastmultigather with sc100 and sc1000
cd ICTV
mkdir sc100
mkdir sc1000

# symlink
ln -s /home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna.k21.zip

# do srun
srun --account=ctbrowngrp -p med2 -J fmg_gb -t 3:00:00 -c 36 --mem=70gb --pty bash

mamba activate branchwater

# run the fastmultigather
cd sc100
sourmash scripts fastmultigather \
../../votu_sigs_s100.txt \
../vmr_MSL38_v1.dna.k21.zip \
-c 48 -k 21 -t 0 -s 100


In [ ]:
# genbank
# run the fastmultigather

cd sc100
sourmash scripts fastmultigather \
../../votu_sigs_s100.txt \
../vmr_MSL38_v1.dna.k21.zip \
-c 48 -k 21 -t 0 -s 100


## redo with rocksdb, in the folder where the symlink is
srun --account=ctbrowngrp -p med2 -J fmg_gb -t 6:00:00 -c 4 --mem=100gb --pty bash
mamba activate branchwater
/usr/bin/time -v sourmash scripts fastmultigather \
votu_sigs_00.txt \
genbank.2023-05.viral.dna-k21-sc100.rocksdb \
-c 4 -k 21 -t 300 -s 100 -o votus.x.genbank.csv

## Rocksdb taking a lot of mem
- try with a zip instead of a txt file
- use the time command to track amount of mem it needs

Opened an issue:
https://github.com/sourmash-bio/sourmash_plugin_branchwater/issues/273

In [ ]:
# split the vOTU sigs into smaller files, maybe that helps
split -l 1000 -d --additional-suffix=.txt ../votu_sigs_s100.txt votu_sigs_

In [ ]:
# try rocksdb again, give it like 300GB, increase treshold, no threads param
srun --account=ctbrowngrp -p high2 -J fmg_gb -t 24:00:00 -c 4 --mem=300gb --pty bash

cd /home/ntpierce/2023-spillover/output.genbank-viral/ 
mamba activate branchwater
/usr/bin/time -v sourmash scripts fastmultigather \
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/signatures_concat/vOTUs.k21.s100.zip \
genbank.2023-05.viral.dna-k21-sc100.rocksdb -k 21 -s 100 -o \
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/viral_taxonomy/genbank/votus.x.genbank.csv -t 300

In [ ]:
# for against a protein db we will need protein predictions
srun --account=ctbrowngrp -p bmm -J prodigal -t 3:00:00 -c 24 --mem=30gb --pty bash
# in ./virsorter2/
mamba activate prodigal
for f in *.fa
do 
prodigal -i $f -a ../../vOTUs_prot/${f%.fa*}.faa -p meta -q
done

# Then sourmash sketch the files at k=10, scaled=100
for f in *.faa
do
sourmash sketch protein -p k=7,scaled=100,noabund,k=10,scaled=100,noabund $f \
-o ../prot_sketches/votus/${f%.faa*}.sig.gz
done


module load parallel
mamba activate branchwater

for f in *.faa
do
echo sourmash sketch protein -p k=7,scaled=100,noabund,k=10,scaled=100,noabund $f \
--name ${f%.faa*} -o ../prot_sketches/votus/${f%.faa*}.sig.gz
done | parallel -j 24

In [ ]:
srun --account=ctbrowngrp -p med2 -J fmg_gb -t 1:00:00 -c 4 --mem=50gb --pty bash

# concat all sigs to a zip
sourmash sig cat *.sig.gz -o ../signatures_concat/vOTUs.k21.s100.zip

# symlink
ln -s ../../sig_files/signatures_concat/vOTUs.k21.s100.zip .

# use these
mamba activate branchwater
for f in /group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/viral_taxonomy/genbank/votu_sigs_00.txt
do
echo $f
sourmash scripts fastmultigather \
$f \
genbank.2023-05.viral.dna-k21-sc100.rocksdb \
-c 4 -k 21 -t 1000 -s 100 \
-o /group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/viral_taxonomy/genbank/votu_sigs_00.x.genbank.csv
done


/usr/bin/time -v sourmash scripts fastmultigather \
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/viral_taxonomy/genbank/votu_sigs_00.txt \
genbank.2023-05.viral.dna-k21-sc100.rocksdb -c 4 -k 21 -s 100 -o \
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/viral_taxonomy/genbank/votus.x.genbank.csv -t 1000

/usr/bin/time -v 

In [ ]:
# sourmash tax
sourmash tax genome --gather-csv <gather_csv> [ ... ] --taxonomy-csv

## For using a rocksdb:
- Symlink the rocksdb directly to the folder youre running fmg in
- Or run the command from the folder that the rocksdb lives in 


## vCONTACT2
- vContact2 is no longer supported, so current installations give bugs in python code
- Java installation doesnt work, maybe try --vcs-mode MCL instead of the java option

In [ ]:
# because vcontact isnt updated with current python, use Chris old version
source /home/csantosm/initconda
conda activate VCONTACT2

# Compare to vcontact2 output
# prodigal file already avail
# remove spaces from the headers..
sed '/^>/ s/ .*//' ../../../virsorter2/protein_files >> tomato_pigeon_br_ns.faa


# and gene2genome
mamba activate vcontact2

python vcontact_gene2genome.py -p 240214_allvOTUs_highq.ns.faa -o \
240214_allvOTUs_highqx.csv -s Prodigal-FAA

# run vcontact2, 24 threads, 70GB, may need more
srun --account=ctbrowngrp -p bmm -J vcontact -t 16:00:00 -c 36 --mem=70gb --pty bash


vcontact2 --raw-proteins 240214_allvOTUs_highq.ns.faa \
--rel-mode 'Diamond' \
--db 'ProkaryoticViralRefSeq85-Merged' \
--proteins-fp 240214_allvOTUs_highq.csv \
--pcs-mode MCL \
--vcs-mode ClusterONE \
--threads 24 \
--c1-bin /home/csantosm/miniconda3/bin/cluster_one-1.0.jar \
--output-dir .

# if vcontact fails, try with clusterOne (in vcontact2_redo):
vcontact2 --threads 24 \
--pcs vcontact2/vConTACT_pcs.csv \
--contigs vcontact2/vConTACT_contigs.csv \
--pc-profiles vcontact2/vConTACT_profiles.csv \
--vcs-mode ClusterONE \
--c1-bin /home/csantosm/miniconda3/bin/cluster_one-1.0.jar \
--db 'ProkaryoticViralRefSeq85-Merged' --output-dir vcontact2_redo

# if vcontact fails, try with MCL (in vcontact2_redp):
vcontact2 --threads 36 \
--pcs vcontact2/vConTACT_pcs.csv \
--contigs vcontact2/vConTACT_contigs.csv \
--pc-profiles vcontact2/vConTACT_profiles.csv \
--vcs-mode MCL \
--c1-bin /home/csantosm/miniconda3/bin/cluster_one-1.0.jar \
--db 'ProkaryoticViralRefSeq85-Merged' --output-dir vcontact2_mcl



### genomad
- Uses ICTV taxonomy (vmr 19)
- I think we only need genomad aggregated-classification


In [ ]:
# db in the emersonlab, symlink that and the vOTU file:
ln -s /group/jbemersogrp/databases/genomad/genomad_db .

# srun it, needs quite some mem
srun --account=ctbrowngrp -p bmm -J genomad -t 1:00:00 -c 6 --mem=20gb --pty bash

# end to end for everything, need to annotate for classification
mamba activate genomad
genomad end-to-end --threads 6 --enable-score-calibration \
200bp.fa genomad_out_200bp genomad_db --splits 3



In [ ]:
# sketch vOTUs at different scales if neccesary 
# (not neccesary cause the scaled 100 should be able to scale up to 1000)
for f in *.fa
do
echo sourmash sketch dna -p k=21,scaled=100 $f --name ${f%.fa*} -o ../../vOTUs_100/${f%.fa*}.sig.gz
done | parallel -j 42
&&
for f in *.fa
do
echo sourmash sketch dna -p k=21,scaled=1000 $f --name ${f%.fa*} -o ../../vOTUs_1000/${f%.fa*}.sig.gz
done | parallel -j 42

In [ ]:
# use snakemake file
srun --account=ctbrowngrp -p med2 -J gathertax -t 5:00:00 -c 48 --mem=40gb --pty bash
snakemake --resources mem_mb=40000 --rerun-triggers mtime -c 48 --rerun-incomplete -k --latency-wait 1
